In [10]:
import collections

import folium
import geopy.distance
import numpy
import pandas
from ortools.sat.python import cp_model

In [11]:
df_sitios_bog = pandas.read_csv("./data/sitios-de-interes-en-bogota.csv", sep=";")
df_sitios_bog = df_sitios_bog[["Nombre", "Geo Point"]]
df_sitios_bog["Lat"] = df_sitios_bog.apply(lambda row: float(row["Geo Point"].split(",")[0]), axis=1)
df_sitios_bog["Lng"] = df_sitios_bog.apply(lambda row: float(row["Geo Point"].split(",")[1]), axis=1)
df_sitios_bog.drop("Geo Point", axis=1, inplace=True)
df_sitios_bog = df_sitios_bog.head(101)
df_sitios_bog = df_sitios_bog.drop_duplicates(subset="Nombre")
# df_sitios_bog = df_sitios_bog.set_index("Nombre")
df_sitios_bog

,Nombre,Lat,Lng
0,Escultura Nave Espacial,4.615251,-74.071534
1,Estatua Tomas Cipriano de Mosquera,4.597386,-74.076421
2,Estatua Ricardo Palma,4.602078,-74.066768
3,Monumento al Almirante José Prudencio Padilla,4.625450,-74.074206
4,Monumento 21 Ángeles,4.732816,-74.075095
...,...,...,...
96,Laboratorios Spaison Ltda,4.681562,-74.117520
97,Luminex Legrand,4.693044,-74.118457
98,Doña Juana ESP SA,4.499661,-74.143927
99,Avianca,4.645531,-74.099483


In [12]:
lat_mean = df_sitios_bog["Lat"].mean()
lng_mean = df_sitios_bog["Lng"].mean()

bogota_map = folium.Map(location=[lat_mean, lng_mean], zoom_start=12, tiles="cartodbdark_matter")
for idx, row in df_sitios_bog.iterrows():
    if idx == "Bavaria SA.":
        folium.CircleMarker([row["Lat"], row["Lng"]], radius=5, color="yellow", fill=True, fill_color="white", fill_opacity=0.6, popup=idx).add_to(bogota_map)
    else:
        folium.CircleMarker([row["Lat"], row["Lng"]], radius=5, color="blue", fill=True, fill_color="white", fill_opacity=0.8, popup=idx).add_to(bogota_map)

bogota_map

#### **Conjuntos**
$$ \text{CLIENTES} = \{1, 2, \ldots, N\} $$
$$ \text{NODOS} = \{0\} \cup \text{CLIENTES} $$

In [13]:
NODOS = df_sitios_bog.index.to_list()
CLIENTES = df_sitios_bog[df_sitios_bog.index != "Bavaria SA."].index.to_list()

#### **Parámetros**
$$ \text{DISTANCIAS}_{n1, n2}: \text{Distancia euclidiana entre } n1 \in \text{NODOS} \text{ y } n2 \in \text{NODOS} $$

In [14]:
df_distancias = pandas.DataFrame()
for nodo1 in NODOS:
    for nodo2 in NODOS:
        df_distancias.loc[nodo1, nodo2] = geopy.distance.geodesic((df_sitios_bog.loc[nodo1, "Lat"], df_sitios_bog.loc[nodo1, "Lng"]), (df_sitios_bog.loc[nodo2, "Lat"], df_sitios_bog.loc[nodo2, "Lng"])).km

C:\Users\sejo_\AppData\Local\Temp\ipykernel_30404\3946343016.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_distancias.loc[nodo1, nodo2] = geopy.distance.geodesic((df_sitios_bog.loc[nodo1, "Lat"], df_sitios_bog.loc[nodo1, "Lng"]), (df_sitios_bog.loc[nodo2, "Lat"], df_sitios_bog.loc[nodo2, "Lng"])).km


In [15]:
model = cp_model.CpModel()

#### **Variables de Decisión**
$$ x_{n1, n2} = \begin{cases} 
1 & \text{Si el vehículo viaja de } n1 \in \text{NODOS} \text{ a } n2 \in \text{NODOS} \\ 
0 & \text{d.l.c.} 
\end{cases} $$

In [16]:
# x = {(nodo1, nodo2): model.new_bool_var(f"x_{nodo1}_{nodo2}") for nodo1 in NODOS for nodo2 in NODOS if nodo1 != nodo2}
x = {(nodo1, nodo2): model.NewBoolVar(f"x_{nodo1}_{nodo2}") for nodo1 in NODOS for nodo2 in NODOS if nodo1 != nodo2}

#### **Función Objetivo**
$$ \text{Minimizar } \sum_{n1 \in \text{NODOS}} \sum_{n2 \in \text{NODOS}} \text{DISTANCIAS}_{n1, n2} \cdot x_{n1, n2} $$

In [17]:
model.Minimize(sum(df_distancias.loc[nodo1, nodo2] * x[nodo1, nodo2] for nodo1 in NODOS for nodo2 in NODOS if nodo1 != nodo2))

#### **Restricciones**

In [18]:
arcs = [(nodo1, nodo2, x[nodo1, nodo2]) for nodo1 in NODOS for nodo2 in NODOS if nodo1 != nodo2]
# model.add_circuit(arcs)
model.AddCircuit(arcs)

In [19]:
solver = cp_model.CpSolver()
solver.parameters.max_time_in_seconds = 20

In [20]:
status = solver.Solve(model)

In [21]:
status == cp_model.OPTIMAL

False

In [22]:
solver.ObjectiveValue()

165.05659312892286

In [23]:
solver.Value(x[92, 37])

1

In [24]:
ruta = []
depot = 92
nodo_actual = depot  # Nodo inicial
while True:
    ruta.append(nodo_actual)
    siguiente_nodo = None
    for nodo2 in NODOS:
        if nodo_actual != nodo2 and solver.Value(x[nodo_actual, nodo2]):
            siguiente_nodo = nodo2
            break
    if siguiente_nodo == depot:  # Si el siguiente nodo es el nodo inicial, terminamos el ciclo
        ruta.append(siguiente_nodo)
        break
    nodo_actual = siguiente_nodo

print("Ruta:", ruta)

Ruta: [92, 37, 89, 56, 31, 84, 24, 44, 10, 58, 38, 49, 36, 69, 85, 40, 29, 20, 14, 12, 98, 94, 11, 30, 47, 87, 50, 93, 18, 39, 53, 64, 33, 66, 1, 7, 6, 5, 2, 57, 0, 43, 83, 82, 35, 34, 60, 22, 91, 17, 51, 3, 65, 19, 59, 21, 86, 28, 80, 100, 77, 16, 54, 99, 90, 63, 62, 61, 23, 48, 78, 55, 27, 79, 71, 42, 70, 96, 97, 41, 32, 72, 67, 8, 4, 74, 52, 13, 25, 45, 81, 76, 9, 75, 73, 68, 88, 15, 95, 46, 26, 92]


In [25]:
def get_bearing(p1, p2):
    long_diff = numpy.radians(p2.lon - p1.lon)
    lat1 = numpy.radians(p1.lat)
    lat2 = numpy.radians(p2.lat)
    x = numpy.sin(long_diff) * numpy.cos(lat2)
    y = numpy.cos(lat1) * numpy.sin(lat2) - (numpy.sin(lat1) * numpy.cos(lat2) * numpy.cos(long_diff))
    bearing = numpy.degrees(numpy.arctan2(x, y))
    if bearing < 0:
        return bearing + 360
    return bearing


def get_arrows(locations, color="black", size=4, n_arrows=3, weight_arrow=3):
    Point = collections.namedtuple("Point", field_names=["lat", "lon"])
    p1 = Point(locations[0][0], locations[0][1])
    p2 = Point(locations[1][0], locations[1][1])
    rotation = get_bearing(p1, p2) - 90
    arrow_lats = numpy.linspace(p1.lat, p2.lat, n_arrows + 2)[1 : n_arrows + 1]
    arrow_lons = numpy.linspace(p1.lon, p2.lon, n_arrows + 2)[1 : n_arrows + 1]
    arrows = []
    for points in zip(arrow_lats, arrow_lons):
        arrows.append(
            folium.RegularPolygonMarker(
                location=points,
                color=color,
                weight=weight_arrow,
                fill=True,
                fill_color=color,
                fill_opacity=1,
                number_of_sides=3,
                radius=size,
                rotation=rotation,
            )
        )
    return arrows

In [27]:
lat_mean = df_sitios_bog["Lat"].mean()
lng_mean = df_sitios_bog["Lng"].mean()

bogota_map = folium.Map(location=[lat_mean, lng_mean], zoom_start=12, tiles="cartodbdark_matter")
for idx, row in df_sitios_bog.set_index("Nombre").iterrows():
    if idx == "Bavaria SA.":
        folium.CircleMarker([row["Lat"], row["Lng"]], radius=10, color="yellow", fill=True, fill_color="white", fill_opacity=0.6, popup=idx).add_to(bogota_map)
    else:
        folium.CircleMarker([row["Lat"], row["Lng"]], radius=5, color="blue", fill=True, fill_color="white", fill_opacity=0.8, popup=idx).add_to(bogota_map)

for i in range(len(ruta) - 1):
    coordinates = [
        df_sitios_bog.loc[ruta[i], ["Lat", "Lng"]].values.tolist(),
        df_sitios_bog.loc[ruta[i + 1], ["Lat", "Lng"]].values.tolist(),
    ]

    color = "gray"
    weight = 2

    pl = folium.PolyLine(coordinates, color=color, weight=weight)
    bogota_map.add_child(pl)

    arrows = get_arrows(locations=coordinates, color=color, size=2, n_arrows=1)
    for arrow in arrows:
        arrow.add_to(bogota_map)


bogota_map